## **Entrega: Exercício Módulo 38**

Utilizei dados da Api do **https://brapi.dev**

VALE3 -> https://brapi.dev/api/quote/VALE3?token=gczpBHkuJNGxCKiDwmCPfb



#### **AWS Lambda Extração** -> m38-def-extract

In [ ]:
import json
import logging
from datetime import datetime


import boto3
import urllib3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:
    
    # -- setup

    URL_VALE3 = 'https://brapi.dev/api/quote/VALE3?token=gczpBHkuJNGxCKiDwmCPfb'
    URL_PETR3 = 'https://brapi.dev/api/quote/PETR3?token=gczpBHkuJNGxCKiDwmCPfb'

    BRONZE_BUCKET = 'm38-bronze'

    client = boto3.client('s3')

    date = datetime.now().strftime('%Y-%m-%d')
    filename = f'stock-exchange-{date}.json'

    try:
        http = urllib3.PoolManager()

        response_vale3 = http.request('GET', URL_VALE3)
        response_vale3 = json.loads(response_vale3.data.decode('utf-8'))

        response_petr3 = http.request('GET', URL_PETR3)
        response_petr3 = json.loads(response_petr3.data.decode('utf-8'))

        logging.info(response_vale3)
        logging.info(response_petr3)
        
    except Exception as exc:
        raise exc


    # -- load

    try:
        with open(f'/tmp/{filename}', mode='w', encoding='utf-8') as fp:
            json.dump([response_vale3, response_petr3], fp)
            fp.close()
        client.upload_file(f'/tmp/{filename}', BRONZE_BUCKET, f'{filename}')
    except ClientError as exc:
        raise exc


    return json.dumps(dict(status=True))
    



#### **AWS Lambda Transformação** -> m38-def-transform

In [ ]:
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError



def lambda_handler(event, context):
    
    # -- setup

    BRONZE_BUCKET = 'm38-bronze'
    SILVER_BUCKET = 'm38-silver'

    client = boto3.client('s3')

    date = datetime.now().strftime('%Y-%m-%d')
    filename_csv = f'stock-exchange-{date}.csv'
    filename_json = f'stock-exchange-{date}.json'

    # -- extract

    client.download_file(BRONZE_BUCKET, filename_json, f'/tmp/{filename_json}')

    with open(f'/tmp/{filename_json}', mode='r', encoding='utf-8') as fp:
        data = json.load(fp)
    
    # -- transform

    vale3 = [data[0]['results'][0][key] for key in ['longName', 'marketCap', 'symbol', 'regularMarketPrice', 
                'regularMarketDayLow', 'regularMarketDayHigh', 'regularMarketVolume']]

    petr3 = [data[1]['results'][0][key] for key in ['longName', 'marketCap', 'symbol', 'regularMarketPrice',
                'regularMarketDayLow', 'regularMarketDayHigh', 'regularMarketVolume']]

    # -- load

    try:
        with open(f'/tmp/{filename_csv}', mode='w', encoding='utf-8') as fp:
            fp.write(','.join([str(x) for x in vale3]) + '\n')
            fp.write(','.join([str(x) for x in petr3]) + '\n')
        
        client.upload_file(f'/tmp/{filename_csv}', SILVER_BUCKET, f'reference_date={date}/{filename_csv}')
    except ClientError as exc:
        raise exc

    
    return json.dumps(dict(status=True))

#### **AWS Lambda Tabela** -> m38-def-table

In [ ]:
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError


def lambda_handler(event, context):

    # -- setup

    SILVER_BUCKET = 'm38-silver'

    query = f'''
    CREATE EXTERNAL TABLE IF NOT EXISTS stock (
        longName string,
        marketCap double,
        symbol string,
        regularMarketPrice float,
        regularMarketDayLow float,
        regularMarketDayHigh float,
        regularMarketVolume bigint
    )
    PARTITIONED BY (
        reference_date string
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
    WITH SERDEPROPERTIES ('separatorChar'=',')
    LOCATION 's3://{SILVER_BUCKET}/'
    '''

    client = boto3.client('athena')

    # -- create

    try:
        client.start_query_execution(
            QueryString=query,
            ResultConfiguration={
                'OutputLocation': f's3://ttheu-query-results/'
            }
        )
    except ClientError as exc:
        raise exc
    

    # --  update

    try:
        client.start_query_execution(
            QueryString='MSCK REPAIR TABLE stock',
            ResultConfiguration={
                'OutputLocation': f's3://ttheu-query-results/'
            }
        )
    except ClientError as exc:
        raise exc
        
    
    return json.dumps(dict(status=True))
    